In [1]:
import datetime
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser, PydanticToolsParser
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI

In [2]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are expert researcher.
            Current time : {time}
            1.{first_instruction}
            2.Reflect and critique your answer. Be severe to maximize improvements.
            3.Recommend search queries to research information and improve your answer.
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Answer the user's question above using the required format.")
    ]
).partial(time = lambda : datetime.datetime.now().isoformat())

In [3]:
llm = AzureChatOpenAI(
        azure_deployment="gpt-4o-mini",  # or your deployment
        api_version="2023-03-15-preview",  # or your api version
        temperature=0)

In [4]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field


class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")
    
    
class AnQ(BaseModel):
    answer : str = Field(description="~250 word detailed answer to the question.")
    reflection : Reflection = Field(description="Your reflection on the initial answer.")
    search_queries : List[str] = Field(
        description = "1-3 search queries for researching improvements to address the critique of your current answer."
    )
    

C:\Users\vinay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
parser = JsonOutputToolsParser(return_id=True)
parser_pydantic = PydanticToolsParser(tools=[AnQ])

In [6]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction = "Provide a detailed ~250 word answer."
)

first_responder = first_responder_prompt_template| llm.bind_tools(
    tools=[AnQ], tool_choice="AnQ"
)



In [7]:
human_msg = HumanMessage(
    content="write about langgraph. Main components of langgraph."
    "Please mention usecases as well"
)

chain = first_responder | parser_pydantic

res = chain.invoke(input = {"messages":[human_msg]})
print(res)

[AnQ(answer='LangGraph is a framework designed to facilitate the development and deployment of language models and natural language processing (NLP) applications. Its architecture is built around several key components that work together to streamline the process of building language-based applications.\n\n### Main Components of LangGraph:\n1. **Graph Structure**: At its core, LangGraph utilizes a graph-based structure to represent relationships between different language elements. This allows for efficient querying and manipulation of language data.\n\n2. **Language Models**: LangGraph integrates various pre-trained language models, enabling users to leverage state-of-the-art NLP capabilities without needing extensive expertise in machine learning.\n\n3. **APIs and SDKs**: The framework provides robust APIs and software development kits (SDKs) that allow developers to easily interact with the underlying models and graph structures, facilitating seamless integration into existing appli

In [8]:
class ReviseAns(AnQ):
    answer : str = Field(description="~250 word detailed answer to the question.")
    reflection : Reflection = Field(description="Your reflection on the initial answer.")
    search_queries : List[str] = Field(
        description = "1-3 search queries for researching improvements to address the critique of your current answer."
    )

In [9]:
revise_instruction = """
                    Revise your previous answer using the new information.
                    - You should use the previous critique to add important information to your answer.
                    - You MUST include numerical citations in your revised answer to esnure it can be verified.
                    - Add a "References" section to the bottom of your answer (which does not count towards the word limit.)
                    - You should use the previous critique to remove superfluous information from your answer and make sure it is not more than 250 words.
                    """

revisor = actor_prompt_template.partial(
    first_instruction = revise_instruction
)| llm.bind_tools(
    tools=[ReviseAns], tool_choice="ReviseAns"
)

In [10]:
from langchain_core.messages import BaseMessage, ToolMessage, AIMessage
from langgraph.prebuilt import ToolInvocation, ToolExecutor
  

answer = AnQ(
    answer="",
    reflection=Reflection(missing="", superfluous=""),
    search_queries=[
        "LangGraph architecture", "LangGraph components and features", "applications of language models in industry"
    ],
    id = "call_xyz",
)

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

search = TavilySearchAPIWrapper()
tavily_tools = TavilySearchResults(api_wrapper =search, max_results = 5)

tool_executor = ToolExecutor([tavily_tools])

C:\Users\vinay\AppData\Local\Temp\ipykernel_8144\3995445223.py:7: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor([tavily_tools])


In [12]:
from collections import defaultdict
import json

def execute_tools(state : List[BaseMessage]) -> List[ToolMessage]:
    tools_invocation : AIMessage = state[-1]
    parsed_tool_calls = parser.invoke(tools_invocation)
    
    ids = []
    tool_invocations = []
    
    for parsed_call in parsed_tool_calls:
        for query in parsed_call["args"]["search_queries"]:
            tool_invocations.append(ToolInvocation(
                tool = "tavily_search_results_json",
                tool_input = query,
            ))
            ids.append(parsed_call["id"]) 
            
    outputs = tool_executor.batch(tool_invocations)
    outputs_map = defaultdict(dict)
    for id_, output, invocation in zip(ids, outputs, tool_invocations):
        outputs_map[id_][invocation.tool_input] = output
        
    tool_messages = []
    for id_, mapped_output in outputs_map.items():
        tool_messages.append(ToolMessage(content = json.dumps(mapped_output), tool_call_id=id_))

    return tool_messages

In [13]:
raw_res = execute_tools(
    state=[
        human_msg,
        AIMessage(
            content = "",
            tool_calls = [
                {
                    "name" : AnQ.__name__,
                    "args" : answer.dict(),
                    "id" : "call_xyz"
                }
            ]
        )
    ]
)

C:\Users\vinay\AppData\Local\Temp\ipykernel_8144\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(


In [14]:
from pprint import pprint

raw_res

[ToolMessage(content='{"LangGraph architecture": [{"url": "https://www.datacamp.com/tutorial/langgraph-tutorial", "content": "LangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner. ... Its robust architecture can handle a high volume of interactions and complex workflows, enabling the development ..."}, {"url": "https://github.com/langchain-ai/langgraph", "content": "Overview. LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures ..."}, {"url": "https://langchain-ai.github.io/langgraph/", "content": "Overview. LangGraph is a library for bu

In [15]:
from typing import List
from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph


In [16]:
MAX_ITERATIONS=2
builder = MessageGraph()
builder.add_node("draft", first_responder)
builder.add_node("execute_tools",execute_tools)
builder.add_node("revise", revisor)
builder.add_edge("draft", "execute_tools")
builder.add_edge("execute_tools", "revise")


In [17]:
def event_loop(state:List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"

In [18]:
builder.add_conditional_edges("revise", event_loop)
builder.set_entry_point("draft")

In [19]:
from mermaid import Mermaid
graph = builder.compile()
Mermaid(graph.get_graph().draw_mermaid())

In [20]:
res = graph.invoke("Write a instagram post caption for trip photos from meghalaya vacation")

C:\Users\vinay\AppData\Local\Temp\ipykernel_8144\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(
C:\Users\vinay\AppData\Local\Temp\ipykernel_8144\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(
C:\Users\vinay\AppData\Local\Temp\ipykernel_8144\3970788705.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_invocations.append(ToolInvocation(


In [21]:
print("Whole Result Object -", res,"--------------------\n\n\n")
print(res[-1].tool_calls[0]["args"]["answer"])

Whole Result Object - [HumanMessage(content='Write a instagram post caption for trip photos from meghalaya vacation', additional_kwargs={}, response_metadata={}, id='b81750ce-50ef-457e-b109-d7ed5fa892f0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3SGNEg3PPtNF1w749Qw0DiPg', 'function': {'arguments': '{"answer":"🌄✨ Just returned from an unforgettable adventure in Meghalaya! From the breathtaking hills to the mesmerizing waterfalls, every moment was a postcard come to life. 💚💦 Exploring the living root bridges was like stepping into a fairy tale, and the vibrant local culture added so much richness to our journey. 🏞️🌿 Can\'t wait to share more memories from this beautiful state! #MeghalayaDiaries #NatureLovers #TravelGoals #Wanderlust","reflection":{"missing":"The caption could include more personal anecdotes or specific experiences to make it more relatable and engaging.","superfluous":"Some emojis may be excessive; a more streamlined use could enhance readabi